In [1]:
import numpy as np
import torch
torch.__version__

'1.8.1+cu102'

## 1. Создаем искусственный нейрон
Реализуем математическую модель искусственного нейрона на основе формулы:
<img src="images/neuron_formula.png">

In [2]:
# Input size:
n = 10

# Inputs:
x = torch.empty((n), dtype=torch.float32)

# Weights:
w = torch.empty(x.shape, dtype=torch.float32)

# Bais:
b = torch.ones(x.shape, dtype=torch.float32)

# Get output:
y = torch.sum(x*w + b)

# Add nonlinearity:
y = torch.sign(y)

### 1.1 Инициализируем тензоры случайными значениями, меняем функцию активации:

In [76]:
# Input size:
n = 10

# Inputs:
x = torch.rand((n), dtype=torch.float32)
x = x * 2.0 - 1.0

# Weights:
w = torch.rand(x.shape, dtype=torch.float32)
w = w * 2.0 - 1.0

# Bais:
b = torch.ones(x.shape, dtype=torch.float32)
b *= 0.1

# Get output:
y = torch.sum(x*w + b)

# ReLU (the rectified-linear activation function ):
y = torch.relu(y)
y

tensor(1.5585)

### 1.2 Перемещаем тензоры на GPU:

In [80]:
# Choose device (CUDA or CPU):
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Input size:
n = 10

# Inputs:
x = torch.rand((n), dtype=torch.float32)
x = x.to(device=device)
x = x * 2.0 - 1.0

# Weights:
w = torch.empty(x.shape, dtype=torch.float32)
w = w.to(device=device)
w = w * 2.0 - 1.0

# Bais:
b = torch.ones(x.shape, dtype=torch.float32)
b = b.to(device=device)
b *= 0.1

# Get output:
y = torch.sum(x*w + b)

# Add nonlinearity (ReLU activation):
y = torch.relu(y)

# CPU -> GPU:
y = y.cpu().numpy()

## 2. Создаем полносвязную нейронную сеть
Реализуем простую полносвязную нейронную сеть:
<img src="images/nn_picture.png">

In [85]:
import torch.nn as nn               # imports base class for NN
import torch.nn.functional as F     # imports activation functions

class MyNet(nn.Module):
    def __init__(self, in_size, out_size):
        super(MyNet, self).__init__()
        # Describe layers:
        self.input = nn.Linear(in_size, 100)
        self.fc1 = nn.Linear(100, 1000)
        self.dropout1 = nn.Dropout(0.25)
        self.fc2 = nn.Linear(1000, 100)
        self.dropout2 = nn.Dropout(0.5)
        self.output = nn.Linear(100, out_size)

    # Forward propagation:
    def forward(self, x):
        '''
        x reprenets our input data
        '''
        # Pass data through layers:
        x = self.input(x)
        x = F.relu(x)
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)

        x = self.output(x)
        
        # Apply softmax to x:
        y = F.log_softmax(x, dim=1)
        
        return y

### 2.1 Инициализируем нейронную сеть и пропускаем через нее батч рандомных данных

In [103]:
# Init a model:
my_nn = MyNet(5, 10)              # input size: 5, output size: 10

# Input batch of data (random):
batch_data = torch.rand((64, 5))  # batch size: 64

# Pass data through the model:
logits = my_nn(batch_data)

# Pick best probability after softmax:
preds = torch.argmax(logits, dim=1)

print (preds)

tensor([6, 2, 2, 2, 6, 2, 6, 6, 2, 2, 6, 2, 6, 7, 1, 2, 6, 2, 6, 6, 6, 7, 2, 2,
        6, 6, 2, 6, 7, 2, 6, 2, 7, 6, 2, 7, 7, 6, 7, 6, 6, 6, 2, 2, 1, 7, 7, 6,
        2, 2, 7, 7, 6, 2, 6, 7, 6, 7, 7, 2, 7, 6, 6, 2])


### 2.2 Делаем то же что и в 2.1, но на GPU:

In [105]:
# Choose device (CUDA or CPU):
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Init a model:
my_nn = MyNet(5, 10)              # input size: 5, output size: 10
my_nn.to(device)

# Input batch of data (random):
batch_data = torch.rand((64, 5))  # batch size: 64
batch_data = batch_data.to(device)

# Pass data through the model:
logits = my_nn(batch_data)

# Pick best probability after softmax:
preds = torch.argmax(logits, dim=1)

# CPU -> GPU:
preds = preds.cpu()

print (preds)

tensor([7, 5, 1, 7, 1, 7, 7, 7, 7, 1, 1, 7, 7, 5, 7, 7, 7, 5, 1, 5, 5, 7, 7, 7,
        7, 5, 7, 3, 1, 1, 7, 5, 7, 5, 7, 7, 7, 5, 5, 7, 7, 5, 7, 5, 5, 5, 5, 7,
        5, 1, 7, 7, 5, 7, 7, 7, 7, 7, 7, 7, 7, 1, 7, 5])


## 3. Создаем сверточную нейронную сеть
Реализуем простую сверточную нейронную сеть:
<img src="images/cnn.jpg">

In [95]:
import torch.nn as nn               # imports base class for NN
import torch.nn.functional as F     # imports activation functions

class CnnNet(nn.Module):
    def __init__(self, out_size):
        super(CnnNet, self).__init__()
        # Describe layers:
        self.conv1 = nn.Conv2d(1, 32, 3, 1)     # 1: in channels, 32: out channels, 3: kernel size, 1: stride
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)         # 9216: in channels, 128: out channels
        self.fc2 = nn.Linear(128, out_size)

    # Forward propagation:
    def forward(self, x):
        '''
        x reprenets our input data
        '''
        # Pass data through conv's layers:
        x = self.conv1(x)
        x = F.relu(x)

        x = self.conv2(x)
        x = F.relu(x)

        # Run max pooling over x:
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        
        # Flatten x with start_dim=1
        x = torch.flatten(x, 1)
        
        # Pass data through FC's layers:
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)

        # Apply softmax to x:
        output = F.log_softmax(x, dim=1)
        
        return output

### 3.1 Инициализируем  сеть и пропускаем через нее батч рандомных данных

In [96]:
# Choose device (CUDA or CPU):
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Init a model:
my_cnn = CnnNet(10)              # output size: 10
my_cnn.to(device)

# Input batch of random 28x28 images:
batch_data = torch.rand((64, 1, 28, 28))  # batch size: 64
batch_data = batch_data.to(device)

# Pass data through the model:
logits = my_cnn(batch_data)

# Pick best probability after softmax:
preds = torch.argmax(logits, dim=1)

# CPU -> GPU:
preds = preds.cpu()

print (preds)

tensor([5, 3, 5, 7, 9, 9, 5, 5, 9, 7, 5, 5, 9, 9, 9, 5, 7, 5, 9, 5, 7, 6, 7, 7,
        9, 9, 7, 9, 9, 7, 5, 9, 5, 9, 9, 9, 7, 5, 9, 7, 5, 9, 9, 7, 5, 5, 6, 6,
        5, 9, 7, 9, 6, 6, 9, 5, 9, 9, 9, 7, 9, 9, 5, 9])


# Нейронная сеть создана, теперь нужно лишь ее обучить! 